In [4]:
GPU_POWER = 280  # Watts (NVIDIA Quadro RTX 8000 actively cooled) / Leistung


def calc_gpu_power_usage(hours):  # Energieverbrauch
    """Calculate the total power usage of the GPU in kWh."""
    return (GPU_POWER * hours) / 1000  # kWh


def calc_seconds_to_hours(seconds):
    """Convert seconds to hours."""
    return seconds / 3600


def calc_cost(power_usage_kwh, cost_per_kwh=0.3969):  # Kosten in Euro/kWh
    """Calculate the cost of running the GPU based on power usage and cost per kWh.

    BDEW meldet für Mai 2025 Durchschnittspreis von 39,69 Cent pro kWh in Deutschland.
    """
    return power_usage_kwh * cost_per_kwh


def calc_emissions(power_usage_kwh, emissions_factor=0.363):  # kg CO2-Emissionen
    """Calculate the CO2 emissions based on power usage and emissions factor.

    In D im Jahr 2024 pro kWh Stromverbrauch 363 g CO2-Äquivalent emittiert.
    """
    return power_usage_kwh * emissions_factor


def dollar_to_euro(dollar_amount, exchange_rate=0.87):
    """Convert USD to EUR using the current exchange rate.

    Exchange rate 19.6.2025 1 USD = 0.87 EUR.
    """
    return dollar_amount * exchange_rate

In [12]:
from pathlib import Path

import pandas as pd

for file in Path("env/output").glob("*.csv"):
    if "extracted" in file.name and "gpt-" not in file.name:
        df = pd.read_csv(file)
        df["duration_h"] = df["duration"].apply(calc_seconds_to_hours)
        df["power_usage_kwh"] = df["duration_h"].apply(calc_gpu_power_usage)
        df["cost_eur"] = df["power_usage_kwh"].apply(calc_cost)
        df["emissions_kg"] = df["power_usage_kwh"].apply(calc_emissions)
        del df["duration_h"]

        new_file_name = file.name.replace("extracted", "processed")
        df.to_csv(f"env/output/{new_file_name}", index=False)


In [5]:
from pathlib import Path

import pandas as pd

# https://platform.openai.com/docs/pricing
# $ per 1M tokens (input, output)
PRICING = {
    "gpt-4.1-nano-2025-04-14": (0.1, 0.4),
    "gpt-4.1-mini-2025-04-14": (0.4, 1.6),
    "gpt-4.1-2025-04-14": (2.0, 8.0),
    "gpt-4o-2024-08-06": (2.5, 10.0),
    "gpt-4o-mini": (0.15, 0.6),
}


def find_pricing(file_name):
    """Find the pricing for the model based on the file name."""
    for model, (input_price, output_price) in PRICING.items():
        if model in file_name:
            return input_price, output_price
    raise ValueError(f"No valid OpenAI model not found in file name: {file_name}")


for file in Path("env/output").glob("*.csv"):
    if "extracted" in file.name and "gpt-" in file.name:
        input_price, output_price = find_pricing(file.name)
        df = pd.read_csv(file)
        df["prompt_tokens_m"] = df["prompt_tokens"] / 1_000_000
        df["completion_tokens_m"] = df["completion_tokens"] / 1_000_000
        df["prompt_cost_usd"] = df["prompt_tokens_m"].apply(lambda x: x * input_price)
        df["completion_cost_usd"] = df["completion_tokens_m"].apply(lambda x: x * output_price)
        df["total_cost_usd"] = df["prompt_cost_usd"] + df["completion_cost_usd"]
        df["total_cost_eur"] = df["total_cost_usd"].apply(dollar_to_euro)
        del df["prompt_tokens_m"]
        del df["completion_tokens_m"]
        del df["prompt_cost_usd"]
        del df["completion_cost_usd"]
        del df["total_cost_usd"]

        new_file_name = file.name.replace("extracted", "processed")
        df.to_csv(f"env/output/{new_file_name}", index=False)